# Restaurant Tips: Which variables — payer’s gender, smoking status, day of the week, time of day, and party size — are associated with the tip percentage?

Group name: 2 / Student names: Rafa Africa, Ben Ni, Yingming Sha, Michael Tham

Course: STAT 301 Section 201 / Date: 04/06/2025 

## (1) Introduction

Tipping is a widespread social norm in service industries, particularly in restaurants, where it can contribute significantly to employee income. However, tipping behavior is not entirely uniform—numerous studies have found that factors such as customer demographics, situational context, and group dynamics can all influence how much people tip (Jewell, 2008). Understanding what drives tipping behavior is valuable not only for economic modeling, but also for helping restaurant managers and servers improve their service strategies and expectations.

This study explores whether variables such as payer’s gender, smoking status, day of the week, time of day, and party size are associated with the tip percentage left in a restaurant setting. Research has shown that, for instance, tipping tends to decrease as party size increases, likely due to a diffusion of responsibility or shared norms within larger groups (Haugom & Thrane, 2022). Similarly, differences in tipping patterns have been observed based on temporal variables like weekday vs. weekend or lunch vs. dinner (Jewell, 2008).

Building upon these insights, this study aims to investigate the relationships between specific variables—payer’s gender, smoking status, day of the week, time of day, and dining party size—and tip percentage in a restaurant setting.

This study contributes to the existing body of knowledge on tipping behavior by focusing on specific variables that have been identified in prior research as potential influencers of tipping practices. By examining these factors collectively, the study aims to provide a comprehensive understanding of their individual and combined effects on tip percentages.

### Research Question
**Our Research Question**: Which variables, payer's gender, smoking status, day of the week, time of day, and dining party size, are associated with the tip percentage in a restaurant?

**Response variable** - tip percentage (total bill divided by the tip amount left by customer);

**Explanatory variables** - sex (gender), day (day of the week), time (Lunch or Dinner), and party size.

**Explanation**: This question is focused on **inference**, as it aims to understand the relationships between the sex, smoker, day, time, party size and the tip percentage. The goal is to identify which factors are significant in determining the tip percentage, rather than predicting individual tip percentage for future transactions.

## (2) Methods and Results

In [ ]:
# First we load the library
library(tidyverse)
library(gridExtra) # for visualization
library(MASS) # for selection
library(car) # for vif
library(httr) # load data
library(rsample) # for train-test split
library(broom) # for glance()

### a) Data 

#### Data Description 

The "Restaurant Tips Dataset" details tipping behavior in a restaurant setting. The dataset was collected over an long period of time and includes a variety of factors that may influence the amount a customer tips. Each entry represents a unique transaction, recording details such as the bill amount, tip given, gender of the payer, smoking preference, day of the week, time of day, and number of diners. This data contains 244 observations and 7 variables. The continuous variables are total_bill, tip, and size, while the categorical variables are sex, smoker, day, and time. 

The dataset used in this study was collected through observational means, meaning the data reflects real-world restaurant transactions without any experimental manipulation. As such, while we can investigate associations between variables and tip percentage, causal conclusions cannot be drawn.

**Description of each variable:**

| id | Key       | Description|
|----|-----------|-----------------------------------------------------------|
|1   |total_bill | Total bill amount in dollars                               |
|2   |tip        | Tip amount in dollars                                      |
|3   |sex        | Gender of the person paying the bill (Male/Female)         |
|4   |smoker     | Whether the person is a smoker (Yes/No)                    |
|5   |day        | Day of the transaction (Thur/Fri/Sat/Sun)                  |
|6   |time       | Time of day when the transaction took place (Lunch/Dinner) |
|7   |size       | Size of the dining party (number of people)                |

**Source and information**

According to the owner of this dataset, Saurabh Badole, the data in this dataset was collected from a popular restaurant over a specified period (Badole, 2024). I retreived this dataset through Badole publishing it on Kaggle.

**Justification**: For this research question, total_bill and tip will be dropped since the tip percentage is calculated based on these two covariates. If we include these two covariates, there will be a multicollinearity issue.

In [ ]:
# Read the data into R
invisible(
GET(
  url = "https://www.kaggle.com/api/v1/datasets/download/saurabhbadole/restaurant-tips-dataset",
  authenticate("yingmingsha", "c52dacb47dd5f58ebfe40f4eb435b1b4", type = "basic"),
  write_disk("restaurant-tips-dataset.zip", overwrite = TRUE)
    )
)

unzip("restaurant-tips-dataset.zip")

tips <- read.csv("tips.csv")

head(tips)

### b) Exploratory Data Analysis (EDA)
- Include 2 effective and creative visualizations : explore the association of some potential explanatory variables with the response (use colours, point types, point size and/or faceting to include more variables) and highlight potential problems (e.g., multicollinearity or outliers)
- Transform some variables if needed and include a clear explanation (e.g. log-transformation may be useful when outliers are present)
- Any summary tables that are relevant to your analysis (e.g., summarize number of observation in groups, indicate if NAs exist)
- Be sure not to print output that takes up a lot of screen space!


In [ ]:
# Main developer: Yingming 

# Check missing values and duplicated rows
summary_table <- tibble(
  Metric = c("Total Missing Values", "Total Duplicate Rows"),
  Count = c(sum(is.na(tips)), sum(duplicated(tips))) 
)
summary_table

In [ ]:
# Main developer: Yingming 

# Clean and wrangle the data into a tidy format
tip_clean <- tips %>%
    distinct() %>% # remove duplicated row
    mutate(tip_per = tip / total_bill * 100) # calculate the bill percentage 

tip_final <- tip_clean %>%
    dplyr::select(-total_bill, -tip) # drop 'total_bill' and 'tip' columns (justification given above)

In [ ]:
# Main developer: Yingming

# Summary tables that are relevant to the analysis
summary_table_sex <- tip_final %>%
  group_by(sex) %>%
  summarize(
      Count = n(),                         # Total count
      Mean = mean(tip_per, na.rm = TRUE)   # Mean
  )

summary_table_smo <- tip_final %>%
  group_by(smoker) %>%
  summarize(
    Count = n(),                                  
    Mean = mean(tip_per, na.rm = TRUE) 
  )

summary_table_day <- tip_final %>%
  group_by(day) %>%
  summarize(
    Count = n(),                               
    Mean = mean(tip_per, na.rm = TRUE)
  )

summary_table_time <- tip_final %>%
  group_by(time) %>%
  summarize(
    Count = n(),                                
    Mean = mean(tip_per, na.rm = TRUE)
  )

summary_table_size <- tip_final %>%
  group_by(size) %>%
  summarize(
      Count = n(),      
      Mean = mean(tip_per, na.rm = TRUE)   
  )

grid.arrange(
  tableGrob(summary_table_sex), tableGrob(summary_table_smo),
  tableGrob(summary_table_day), tableGrob(summary_table_time),
  ncol = 2
)

**Interpretation of Summary Tables:**

Since the dataset contains more categorical variables, the summary tables provide a basic overview of each covariate in relation to our response variable. From the tables, we can see that there is not much difference in the average tip percentage across sex, smoker status, day, and time. However, for party size, there appears to be a potential decrease in the average tip percentage as the party size increases.

In [ ]:
# need 2 visualizations (my previous visualization is about total_bill, does anyone have other visualization?) (!)

In [ ]:
# Scatterplot showing the tip percentages against the total bill
tips_plot <- 
  tips %>%
  ggplot(aes(x = total_bill, y = tip/total_bill, color = sex)) +  
  geom_smooth(method = "lm", se = FALSE, linewidth = 1.5) +  
  geom_point() +
  facet_grid(time ~ day) +
  ggtitle("Tip Amount vs. Total Bill by Time and Day") +
  xlab("Total Bill ($)") +
  ylab("Tip ($)") +
  labs(color = "Sex")  

tips_plot

**The reasons why visualizations are relevant**: 

(!)

### c) Methods: Plan
- Describe in written English the methods you used to perform your analysis from beginning to end.
- Provide a detailed justification of the method(s) used. The analysis must be based primarily on methods learned in the class (other method can be used for comparison).
- Make sure that the analysis responded the question posed and that the proposed method is appropriate for the characteristics of the data.
- If variable selection methods are used, justify the method used and explain what data will be used.
- If various models will be compared explain how you will select a final one.
- Make sure to interpret/explain the results you obtain. It’s not enough to just say, “I fitted a linear model with these covariates, and my R-square is 0.87”: If inference is the aim of your project, a detailed interpretation of your fitted model is required, as well as a discussion of relevant quantities. For example, which coefficient(s) is(are) statistically significant? interpretation of significant coefficients, how does the model fit the data? among other points; If prediction is the project's aim, describe the test data used or how it was created; In both cases, a careful model assessment must be conducted.
- Include no more than 3 visualizations and/or tables to support your points. Ensure your tables and/or figures are labelled with a figure/table number and readable fonts.

Code: all the analysis, from reading the data to visualizing results, must be based on clean, reproducible and well-commented code

**Proposed Method:** Multiple Linear Regression

**Why is this method appropriate?**

This method is suitable because it allows us to model the relationship between a continuous response variable (tip percentage) and multiple predictor variables. The coefficients of the regression model will give us clear insights into how each predictor influences the tip percentage and easy to interpret.

**Which assumptions are required, if any, to apply the method selected?**
1. Linearity: The relationship between the predictors and the response variable is assumed to be linear. 
2. Independence: Observations are assumed to be independent of each other, meaning that the tip percentage of one transaction doesn’t affect others.
3. Homoscedasticity: The variance of the residuals (errors) is assumed to be constant across all levels of the predictors.
4. Normality of Residuals: The residuals are assumed to follow a normal distribution.

**What are the potential limitations or weaknesses of the method selected?**
1. Linearity Assumption: If the relationship between the predictors and the tip percentage is non-linear, the model may not perform well. 
2. Multicollinearity: If any of the predictors are highly correlated with each other (e.g., **day** and **time**), it could lead to multicollinearity, where it becomes difficult to isolate the effect of individual predictors. However, this can be checked using vif().

**Variable Selection Method:** Stepwise Selection

We will be using stepwise selection to refine our model and identify the most significant predictors of tip percentage. Stepwise selection is useful when there is some uncertainty about which predictors are most relevant, as it allows for both adding and removing variables based on statistical criteria. 

We will apply stepwise selection using AIC, which balances model fit and complexity. The response variable is tip percentage, and the predictors initially considered were sex, smoker status, day, time, and size.

#### Computational Code and Output

In [ ]:
# Main developer: Yingming 

tip_full_model <- lm(tip_per ~ ., data = tip_final)

# Check multicollinearity in the full Model
vif(tip_full_model) 

summary(tip_full_model)

**Initial Analysis:**

Based on the VIF, `day` and `time` exhibit multicollinearity. From the summary(), we observe the following:

- sexMale: Coefficient = -0.87 (p = 0.299), suggesting that men tip 0.87% less than women. However, this result is not statistically significant.
- smokerYes: Coefficient = 0.39 (p = 0.648), indicating that smokers tip 0.39% more than non-smokers. But this effect is also not statistically significant.
- day (reference: Friday): There are no statistically significant differences in tipping between days (Saturday, Sunday, Thursday), based on the p-values.
- timeLunch: Coefficient = 2.34 (p = 0.373), suggesting that lunch customers tip 2.34% more than dinner customers, but this effect is not significant.
- size: Coefficient = -0.97 (p = 0.023), meaning larger parties tip 0.97% less. This result is statistically significant.

Fitting the full model provides some insight into how the predictor variables relate to the response variable. However, to improve the model, we will now apply a variable selection method to identify a more optimal model.

In [ ]:
# Main developer: Yingming 

# Split data into training and testing sets
set.seed(123)  
tip_split <- initial_split(tip_final, prop = 0.7, strata = tip_per)  # 70% train, 30% test
tip_train <- training(tip_split)
tip_test <- testing(tip_split)

# Fit model using training data
# Full model with all covariates
tip_full_train <- lm(tip_per ~ ., data = tip_train)

# Perform stepwise selection
tip_backward <- stepAIC(tip_full_train, direction = "backward", k = 2)

Using stepwise selection, it was found that party size (`size`) is the most important factor influencing tip percentage, while other variables (such as sex, smoker status, day of the week, and time of day) do not contribute meaningfully when party size is accounted for. This finding aligns with our previous interpretation of the full model, where party size was the only significant predictor of tip percentage.

In [ ]:
# Main developer: Yingming 

# Fit the model using the selected variables
tip_select <- lm(tip_per ~ size, data = tip_train)

# Compare the selected model and the full model using ANOVA
anova(tip_select, tip_full_train)

**Interpretation of the result**:

According to the ANOVA results, there is no significant difference between the full model and the simplified model. The p-value of 0.29 (greater than 0.05) indicates that the full model with the additional predictors does not fit significantly better than the simplified model. Therefore, the reduced model might be sufficient.

In [ ]:
# Main developer: Yingming 

pred_select <- predict(tip_select, newdata = tip_test)
rmse_select <- sqrt(mean((tip_test$tip_per - pred_select)^2))

# Calculate RMSE for the full model
pred_full <- predict(tip_full_train, newdata = tip_test) 
rmse_full <- sqrt(mean((tip_test$tip_per - pred_full)^2))

# Calculate R^2 values
total <- sum((tip_test$tip_per - mean(tip_test$tip_per))^2)
residual_select <- sum((tip_test$tip_per - pred_select)^2)
r2_select <- 1 - (residual_select / total)

residual_full <- sum((tip_test$tip_per - pred_full)^2)
r2_full <- 1 - (residual_full / total)

# Create a data frame to store results
results <- data.frame(
  Model = c("Selected Model", "Full Model"),
  RMSE = c(rmse_select, rmse_full),
  R_2 = c(r2_select, r2_full)
)

# Print the table
print(results)

The selected model has a lower RMSE (4.854) compared to the full model (5.143), indicating better prediction accuracy. The selected model also has a positive R² (0.060), meaning it explains 6% of the variation in the response, while the full model has a negative R² (-0.055), suggesting poor performance. Therefore, the selected model is the better choice.

In [ ]:
# Residual plot
# Calculate residuals for the selected model
residual_select <- tip_test$tip_per - pred_select

# Calculate residuals for the full model
residual_full <- tip_test$tip_per - pred_full

# Residual Plot for Selected Model
plot(pred_select, residual_select, 
     main = "Residual Plot: Selected Model", 
     xlab = "Predicted Values", 
     ylab = "Residuals")

# Residual Plot for Full Model
plot(pred_full, residual_full, 
     main = "Residual Plot: Full Model", 
     xlab = "Predicted Values", 
     ylab = "Residuals")

(help... The graphs contradict the numbers (R² and RMSE). There might be something wrong with my code???)

Residual Plot: Selected Model
There's noticeable non-random pattern, which suggests heteroscedasticity (non-constant variance).

Residual Plot: Full Model
The points look more randomly scattered, which is ideal.

The Full Model's residuals appear more random and evenly distributed, which is a key assumption in linear regression (residuals should be randomly scattered with constant variance and mean near zero). The Selected Model violates this by showing strong clustering and signs of heteroscedasticity. 

RMSE and R² suggest the selected model is better for prediction accuracy.

Residual plots suggest the full model might be more appropriate in terms of assumptions.

### (3) Discussion (!)
In this section, you’ll interpret and reflect on the results you obtained in the previous section with respect to the main question/goal of your project.
- Summarize what you found and the implications/impact of your findings
- If relevant, discuss whether your results were what you expected to find
- Discuss how your model could be improved


- Discuss future questions/research this study could lead to
This study opens the door to several potential avenues for future research. One major limitation of the current analysis is the lack of variables related to service quality, customer income level, or restaurant type, all of which could meaningfully influence tipping behavior. Future studies could investigate how interpersonal dynamics, such as server friendliness or attentiveness, interact with customer demographics to affect tipping. Additionally, incorporating longitudinal or transactional data could help explore temporal patterns in tipping, such as whether tipping behavior changes over time or during holidays. Expanding this analysis to a larger, more diverse sample across different cities or restaurant chains could also provide insight into how regional or cultural factors shape tipping norms. Finally, if experimental or quasi-experimental data were available, future work could attempt to move beyond inference toward causal analysis, exploring whether targeted interventions (e.g., menu messaging or suggested tips) actually influence customer tipping decisions.

### (4) References (!)

Badole, S. (2021). Restaurant Tips Dataset. Kaggle. https://www.kaggle.com/datasets/saurabhbadole/restaurant-tips-dataset/data

Haugom, E., & Thrane, C. (2022). The Effects of Party Size and Dining Time on Restaurant Patrons’ Tipping Behavior: Results from a Large-Scale Transaction Database. Retrieved from https://papers.ssrn.com/sol3/Delivery.cfm/SSRN_ID4101280_code5225184.pdf?abstractid=4101280&mirid=1

Jewell, C. N. (2008). Factors Influencing Tipping Behavior in a Restaurant. Psi Chi Journal of Undergraduate Research, 13(1), 38–48. Retrieved from https://cdn.ymaws.com/www.psichi.org/resource/resmgr/journal_2008/Spring08JNJewell.pdf